In [ ]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, callbacks
import keras_tuner as kt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
X_train_scaled = np.load("X_train_scaled.npy")
X_all_scaled = np.load("X_all_scaled.npy")
y = np.load("labels.npy")
input_dim = X_train_scaled.shape[1]

In [ ]:
def build_autoencoder_sequential(hp):
    units1 = hp.Int('units1', min_value=64, max_value=512, step=64)
    units2 = hp.Int('units2', min_value=32, max_value=256, step=32)
    latent = hp.Int('latent', min_value=8, max_value=max(8, input_dim // 4), step=4)
    l1_strength = hp.Float('l1', 1e-6, 1e-3, sampling='log')
    lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')

    model = keras.Sequential([
        layers.InputLayer(input_shape=(input_dim,)),
        layers.GaussianNoise(0.1),

        layers.Dense(units1, activation=None, activity_regularizer=regularizers.l1(l1_strength)),
        layers.LeakyReLU(alpha=0.1),

        layers.Dense(units2, activation=None, activity_regularizer=regularizers.l1(l1_strength)),
        layers.LeakyReLU(alpha=0.1),

        layers.Dense(latent, activation=None, activity_regularizer=regularizers.l1(l1_strength), name="bottleneck"),
        layers.LeakyReLU(alpha=0.1),

        layers.Dense(units2),
        layers.LeakyReLU(alpha=0.1),

        layers.Dense(units1),
        layers.LeakyReLU(alpha=0.1),

        layers.Dense(input_dim, activation="linear")
    ])

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="mse")
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_autoencoder_sequential,
    objective="val_loss",
    max_trials=10,
    directory="kt_dir_seq",
    project_name="ae_unsw_seq",
    overwrite=True
)

es = callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

In [ ]:
print("Starting hyperparameter search...")
tuner.search(
    X_train_scaled, X_train_scaled,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    callbacks=[es],
    verbose=2
)

best_hps = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:", best_hps.values)

Trial 10 Complete [00h 00m 45s]
val_loss: 0.16518457233905792

Best val_loss So Far: 0.11670368909835815
Total elapsed time: 00h 08m 31s
Best hyperparameters: {'units1': 192, 'units2': 224, 'latent': 8, 'l1': 1.5090582916190937e-06, 'lr': 0.0023949500544728688}


In [ ]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    X_train_scaled, X_train_scaled,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)],
    verbose=2
)


In [ ]:
train_recon = model.predict(X_train_scaled, verbose=0)
train_mse = ((train_recon - X_train_scaled) ** 2).mean(axis=1)
threshold = np.percentile(train_mse, 95)


In [ ]:
model.save("models/autoencoder_model.h5")
np.save("data/threshold.npy", threshold)

print(f"Model + threshold saved. Threshold = {threshold:.4f}")